In [1]:
# !pip install nltk
# !pip install accelerate
# !pip install datasets

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import nltk
from nltk.corpus import movie_reviews

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

import torch
from torch import nn

from datasets import load_metric

from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import Trainer, TrainingArguments

In [3]:
nltk.download("movie_reviews")
fieldids = movie_reviews.fileids()
reviews = [ movie_reviews.raw(fieldid) for fieldid in fieldids ]
categories = [ movie_reviews.categories(fieldid) for fieldid in fieldids ]

[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!


In [4]:
# labeled_categories = LabelEncoder().fit_transform(categories)
labeled_categories = LabelEncoder().fit_transform(categories).tolist()
labeled_categories[:5]

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:116: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[0, 0, 0, 0, 0]

In [5]:
X_train, X_test ,y_train, y_test = \
    train_test_split(reviews, labeled_categories, stratify=labeled_categories, random_state=42)

In [6]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

train_inputs = tokenizer(X_train, truncation=True, padding=True, return_tensors='pt')
test_inputs = tokenizer(X_test, truncation=True, padding=True, return_tensors='pt')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [7]:
print( train_inputs['input_ids'].shape, test_inputs['input_ids'].shape )

for item in train_inputs.items():
    print(item[1][2].shape)
    break

torch.Size([1500, 512]) torch.Size([500, 512])
torch.Size([512])


In [8]:
class ReviewDataset(torch.utils.data.Dataset):
    def __init__(self, inputs, labels):
        self.inputs = inputs
        self.labels = labels
        pass

    def __getitem__(self, idx):
        input = { k: torch.tensor(v[idx]) for k, v in self.inputs.items() }
        input['label'] = torch.tensor(self.labels[idx])
        return input

    def __len__(self):
        return len(self.labels)

train_dataset = ReviewDataset(train_inputs, y_train)
test_dataset = ReviewDataset(test_inputs, y_test)

In [9]:
for data in train_dataset:
    print(data.keys())
    # print(data)
    break

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'label'])


<ipython-input-8-4faa0d83da1d>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input = { k: torch.tensor(v[idx]) for k, v in self.inputs.items() }


In [10]:
device_name = "cuda:0" if torch.cuda.is_available() else 'cpu'
device = torch.device(device_name)
device

device(type='cuda', index=0)

In [11]:
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased")

metric = load_metric('accuracy')
def compute_metrics(evaluation_dataset):
    logits, labels = evaluation_dataset
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

train_args = TrainingArguments(
    output_dir="checkpoints",
    num_train_epochs=5,
    evaluation_strategy="steps",
    eval_steps=500,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8
)

trainer = Trainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
    args=train_args
)

trainer.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-11-1e32037e6839>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric('accuracy')
/usr/local/lib/python3.10/dist-packages/datasets/load.py:753: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.17.1/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will b

Step,Training Loss,Validation Loss,Accuracy
500,0.369000,0.686723,0.844000


<ipython-input-8-4faa0d83da1d>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input = { k: torch.tensor(v[idx]) for k, v in self.inputs.items() }


TrainOutput(global_step=940, training_loss=0.21839773502755672, metrics={'train_runtime': 803.9834, 'train_samples_per_second': 9.329, 'train_steps_per_second': 1.169, 'total_flos': 1973332915200000.0, 'train_loss': 0.21839773502755672, 'epoch': 5.0})

In [12]:
torch.cuda.empty_cache()

In [ ]:
model.to(device)
batch_size = 1
all_prediction_labels = []
for idx in range(len(y_test)//batch_size):
    # input_texts = X_train[idx*batch_size:(idx+1)*batch_size]
    input_texts = X_test[idx*batch_size:(idx+1)*batch_size]
    inputs = tokenizer(input_texts, padding=True, truncation=True, return_tensors="pt")
    inputs = inputs.to(device)
    result = model(**inputs) # 예측
    predictions = torch.softmax(result.logits, dim=1)
    prediction_labels = predictions.cpu().detach().numpy().argmax(axis=1)
    all_prediction_labels.extend(prediction_labels.tolist())

accuracy = (np.array(all_prediction_labels) == np.array(y_test)).sum() / len(y_test)
print(f"train accuracy : {accuracy}")

In [22]:
trainer.evaluate(test_dataset)

<ipython-input-8-4faa0d83da1d>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input = { k: torch.tensor(v[idx]) for k, v in self.inputs.items() }


{'eval_loss': 0.7529026865959167,
 'eval_accuracy': 0.882,
 'eval_runtime': 17.4033,
 'eval_samples_per_second': 28.73,
 'eval_steps_per_second': 3.62,
 'epoch': 5.0}